In [67]:
import numpy as np
import nltk
from stop_words import get_stop_words
from nltk.corpus import stopwords
import re
from nltk.stem.porter import *
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt
from collections import Counter
import string
import pandas as pd
import xml.etree.ElementTree as ET

Can't read the XML file

In [68]:
#return list of words
def preprocess(sent):
    stop_words = list(get_stop_words('en'))         #About 900 stopwords
    stemmer = PorterStemmer()
    #tokenization
    words = str(sent).split()
    new_words = []
    for w in words:
        #lowercase
        w = w.lower()
        #remove punctuaction, fast
        table = str.maketrans({key: None for key in string.punctuation})
        w = w.translate(table)
        #remove stopwords, you can change the stopwords list in here
        if w not in stop_words:
            w = stemmer.stem(w)
            new_words.append(w)
        
    return new_words

term_freq and its document freq    

In [69]:
#return doc{word:freq}
def term_freq_line_nick(line):
    newline = preprocess(line)  
    term_freq = {}
    for w in newline:
        if w not in term_freq:
            term_freq[w] = 0
        else:
            term_freq[w] = term_freq[w] + 1
    return term_freq

def term_freq_line(line):
    newline = preprocess(line)  
    term_freq = defaultdict(int)
    for w in newline:
        term_freq[w] += 1
    return term_freq

def count_freq(line):
    wordcount = Counter(preprocess(line))
    return wordcount
    
#term_freq and its document freq    
def term_freq(file):
    term_freq_list = {}
    with open(file) as f:

        id_num = None
        for line in f:
            line = line.strip() # remove beginning and trailing whitespaces
            if "ID: " in line:
                line = line.replace("ID: ", "")
                id_num = str(line)
            else:
                term_freq_list[id_num] = count_freq(line)
                #term_freq_list.append((id_num, term_freq_line(line)))
        return term_freq_list

term_freq('././collections/sample.txt')   

{'1': Counter({'drink': 1, 'like': 2, 'text': 1, 'wink': 1}),
 '2': Counter({'drink': 3, 'like': 1, 'text': 1}),
 '3': Counter({'drink': 1, 'ink': 1, 'like': 1, 'text': 1, 'thing': 1}),
 '4': Counter({'drink': 1, 'ink': 1, 'like': 1, 'pink': 1, 'text': 1}),
 '5': Counter({'drink': 1,
          'ink': 1,
          'like': 1,
          'pink': 1,
          'text': 1,
          'wink': 1})}

get list of documents where this term occured 

get position list

In [71]:
def get_postion(line):
    pos_list = {}
    newline = preprocess(line)
    for position, w in enumerate(newline):   
        #print(w, position)
        if w not in pos_list:
            # pos_list[w] = []
            # pos_list[w].append(position)
            pos_list[w] = [position]
            
        else:
            pos_list[w].append(position)
    return pos_list

        
def postion_list(file):
    with open(file) as f:
        postion_l ={}
        id_num = None
        for line in f:
            line = line.strip() # remove beginning and trailing whitespaces
            if "ID: " in line:
                line = line.replace("ID: ", "")
                id_num = str(line)
            else:
                #term_freq_list[id_num] = count_freq(line)
                postion_l[id_num]= get_postion(line)
        return postion_l

p_l = postion_list('././collections/sample.txt')

In [554]:
def get_inverted_list(line, document_number, pos_ii):
    line = preprocess(line)

    for position, word in enumerate(line):
        if word in pos_ii:
            if document_number in pos_ii[word]:
                pos_ii[word][document_number].append(position)
            else:
                pos_ii[word][document_number] = [position]
        else:
            pos_ii[word] = {document_number:[position]}
    return pos_ii

def produce_ii(file):
    pos_ii = {}
    with open(file) as f:
        postion_l ={}
        id_num = None
        for line in f:
            line = line.strip() # remove beginning and trailing whitespaces
            if "ID: " in line:
                line = line.replace("ID: ", "")
                id_num = str(line)
            else:
                #term_freq_list[id_num] = count_freq(line)
                pos_ii = get_inverted_list(line, id_num, pos_ii)
        return pos_ii

pos_ii = produce_ii('././collections/sample.txt')
print(pos_ii)
print(len(pos_ii['text'].keys()))
 
words = 'pink'
words2 = 'ink'


{'text': {'1': [0], '2': [0], '3': [0], '4': [0], '5': [0]}, 'like': {'1': [1, 3], '2': [1], '3': [2], '4': [2], '5': [1]}, 'wink': {'1': [2], '5': [2]}, 'drink': {'1': [4], '2': [2, 3, 4], '3': [3], '4': [3], '5': [3]}, 'thing': {'3': [1]}, 'ink': {'3': [4], '4': [1], '5': [5]}, 'pink': {'4': [4], '5': [4]}}
5


In [555]:
pos_ii.get('text')

{'1': [0], '2': [0], '3': [0], '4': [0], '5': [0]}

In [660]:
def search_or(words):   
    dic2 = {}
    doc_l = []
    for i in words:
        result = pos_ii.get(i)
        if result is None:
            raise Exception('Word ' + words[i] + ' is not found in the corpora. This behaviour is not implemented yet')
        dic2[i] = result
    #print(dic2)
    #easy version, return documents only
#     print(result)
#     for doc, pos in result.items():
#         if doc not in doc_l:
#             doc_l.append(doc)
#     print(doc_l)
    results_dictionary = {}
    for word, pos_l in dic2.items(): 
        #print(word, pos_l)
        for doc, pos in pos_l.items():
            if doc not in results_dictionary:
                results_dictionary[doc] = {word: pos}
                #results_dictionary[doc] = {}
                #results_dictionary[doc][word] = pos
            else:
                results_dictionary[doc][word] = pos
    #print(results_dictionary)
       
    return results_dictionary
search_or(['like','drink'])

{'1': {'drink': [4], 'like': [1, 3]},
 '2': {'drink': [2, 3, 4], 'like': [1]},
 '3': {'drink': [3], 'like': [2]},
 '4': {'drink': [3], 'like': [2]},
 '5': {'drink': [3], 'like': [1]}}

In [557]:
def search_and(words):
    #return all doc contain the words
    result = {}
    result_or = search_or(words)
    for doc, word_pos in result_or.items():
        if len(word_pos) == len(words):
            if doc not in result:
                result[doc] = word_pos
    return result

search_and(['text', 'ink', 'drink'])
                
#     count = {}
#     for word, doc_pos in result_l.items():
#         for doc,v in doc_pos.items():
#             if doc not in count:
#                 count[doc] = 1
#             else:
#                 count[doc] = count[doc] + 1
#     #print(count)


{'3': {'drink': [3], 'ink': [4], 'text': [0]},
 '4': {'drink': [3], 'ink': [1], 'text': [0]},
 '5': {'drink': [3], 'ink': [5], 'text': [0]}}

In [558]:
words = ['text','drink']
not_word = ['ink']
def search_not(words):
    result_or = search_or(words)
    doc_l = []
    result = []
    for doc, pos_l in result_or.items():
        doc_l.append(doc)
#return all text in result doc


#     for i in range(1,6):
#         if i not in doc_l:
#             result.append(i)
        
#     #get all doc contain key words
#     exist = search_or(words)
#     #get doc list that contain key words
#     doc_l = []
#     for dic in exist:
#         for doc, pos in dic.items():
#             doc_l.append(doc)
#     #return doc that is not in doc list
#     search_not_l=[]
#     for dic,v in pos_ii.items():
#          for doc,pos in v.items():
#                 if doc not in doc_l:
#                     search_not_l.append(doc)
    return doc_l
    
search_not(words)

['1', '2', '3', '4', '5']

phrase search

In [559]:
# return all doc contain keywords
#search_or(['text','ink','drink'])
print(pos_ii.get('text'))
print(pos_ii.get('ink'))

{'1': [0], '2': [0], '3': [0], '4': [0], '5': [0]}
{'3': [4], '4': [1], '5': [5]}


In [666]:

def phrase_search(words):
    result_and = search_and(words)
    result = []
    for doc, word_pos in result_and.items():
        arr1 = result_and[doc][words[0]]
        found = False
        for num in arr1:   #check for proximity
            for i in range(1,len(words)):
                if num + i in result_and[doc][words[i]]:
                    found = True
                else:
                    found = False
                    break
            if found:
                break
        if found:
           result.append(doc)
    return result

phrase_search(['like','drink'])

['1', '2', '3', '4']

In [635]:
##lab 3
def tf_idf(w):
    def Tf(w):
        result = []
        for doc, word_pos in pos_ii[w].items():
            result.append((doc, len(word_pos)))
            result2 = sorted(result, key=lambda x: x[1], reverse=True)
        return result2

    def df():
        result = {}
        for word, doc_pos in pos_ii.items():
            doc_count = 0
            for doc, pos in doc_pos.items():
                doc_count = doc_count + 1
                result[word] = doc_count
        return result

    def get_N():
        count = 0
        for word, doc_pos in pos_ii.items():
            count += 1 
        return count
    
    def idf():
        result = []
        for word, doc_num in df().items():
        #print (word,doc_num)
            idf = np.log10(get_N()/doc_num)
            result.append((word,idf))
        return result
   
#tf_idf
    def final(w):
        term_freq = Tf(w)
        for w1 in idf(): 
            if w1[0] == w:
                w_idf = w1[1]  
        result_l =[]
        for freq in term_freq:
            tfidf_result = (np.log10(freq[1])+1) * w_idf 
            result_l.append((freq[0],tfidf_result))
        return result_l
    
    r = final(w)
    
    return r
 
tf_idf('drink')

[('2', 0.21584882741075853),
 ('1', 0.14612803567823801),
 ('3', 0.14612803567823801),
 ('4', 0.14612803567823801),
 ('5', 0.14612803567823801)]